# Обучение и сохранение моделей (контрольная: TF-IDF, тестовая: BERT)

**Цель ноутбука.**  
Обучить две градиентные модели (CatBoostClassifier) на ранее подготовленных признаках:
- **Контрольная**: TF-IDF -> PCA + календарные/OHE/MTE фичи  
- **Тестовая**: BERT-эмбеддинги (PCA) + календарные/OHE/MTE фичи  

Провести базовую оффлайн-оценку по **ROC-AUC** и сохранить веса в формате **`.cbm`** для последующей загрузки сервисом рекомендаций.

**Входные данные:**
- `X_train_tfidf.csv`, `y_train_tfidf.csv`, `X_test_tfidf.csv`, `y_test_tfidf.csv`
- `X_train_bert.csv`,  `y_train_bert.csv`,  `X_test_bert.csv`,  `y_test_bert.csv`

**Выходные данные / артефакты:**
- `model_control`  (CatBoost, контроль)  
- `model_test`     (CatBoost, тест)

> Примечание: ROC-AUC — прокси-метрика. В проде целевой критерий — **HitRate@5**. Рост ROC-AUC не гарантирует рост HitRate@5 — дополнительно проверяется чекером LMS.


In [ ]:
# Импорты для работы с данными и базовых визуализаций + настройки вывода
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

pd.set_option('display.max_rows', 80)

In [ ]:
# Единый стиль графиков
import matplotlib as mlp

mlp.rcParams['lines.linewidth'] = 5

mlp.rcParams['xtick.major.size'] = 20
mlp.rcParams['xtick.major.width'] = 5
mlp.rcParams['xtick.labelsize'] = 20
mlp.rcParams['xtick.color'] = '#FF5533'

mlp.rcParams['ytick.major.size'] = 20
mlp.rcParams['ytick.major.width'] = 5
mlp.rcParams['ytick.labelsize'] = 20
mlp.rcParams['ytick.color'] = '#FF5533'

mlp.rcParams['axes.labelsize'] = 20
mlp.rcParams['axes.titlesize'] = 20
mlp.rcParams['axes.titlecolor'] = '#00B050'
mlp.rcParams['axes.labelcolor'] = '#00B050'

In [3]:
# Загрузка данных для контрольной модели (TF-IDF)
X_train_tfidf = pd.read_csv('X_train_tfidf.csv')
y_train_tfidf = pd.read_csv('y_train_tfidf.csv')
X_test_tfidf = pd.read_csv('X_test_tfidf.csv')
y_test_tfidf = pd.read_csv('y_test_tfidf.csv')

In [4]:
# Загрузка данных для тестовой модели (BERT)
X_train_bert = pd.read_csv('X_train_bert.csv')
y_train_bert = pd.read_csv('y_train_bert.csv')
X_test_bert = pd.read_csv('X_test_bert.csv')
y_test_bert = pd.read_csv('y_test_bert.csv')

# Тренировка модели и оценка её качества

In [ ]:
# Модель: CatBoostClassifier + метрика для оффлайн-валидации (ROC-AUC)
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
# Обучение контрольной модели на TF-IDF фичах; сохраняем веса в формате .cbm
model_control = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.25,
    depth=6,
    subsample=0.8,
    bootstrap_type="Poisson",
    l2_leaf_reg=3,
    task_type="GPU",
    verbose=False
)
model_control.fit(X_train_tfidf, y_train_tfidf)
model_control.save_model('model_control', format='cbm')

In [ ]:
# Обучение тестовой модели на BERT фичах; сохраняем веса в формате .cbm
model_test = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.25,
    depth=6,
    subsample=0.8,
    bootstrap_type="Poisson",
    l2_leaf_reg=3,
    task_type="GPU",
    verbose=False
)
model_test.fit(X_train_bert, y_train_bert)
model_test.save_model('model_test', format='cbm')

In [ ]:
# Оценка качества контрольной модели (ROC-AUC на train/test)
print("Контрольная модель:")
print(f"Train ROC-AUC: {roc_auc_score(y_train_tfidf, model_control.predict_proba(X_train_tfidf)[:, 1]).round(4)}")
print(f"Test ROC-AUC:  {roc_auc_score(y_test_tfidf, model_control.predict_proba(X_test_tfidf)[:, 1]).round(4)}")

Контрольная модель:
Train ROC-AUC: 0.6896
Test ROC-AUC:  0.6426


In [ ]:
# Оценка качества тестовой модели (ROC-AUC на train/test)
print("Тестовая модель:")
print(f"Train ROC-AUC: {roc_auc_score(y_train_bert, model_test.predict_proba(X_train_bert)[:, 1]).round(4)}")
print(f"Test ROC-AUC:  {roc_auc_score(y_test_bert, model_test.predict_proba(X_test_bert)[:, 1]).round(4)}")

Тестовая модель:
Train ROC-AUC: 0.6957
Test ROC-AUC:  0.6583


In [ ]:
# Проверка корректности загрузки моделей
from_file = CatBoostClassifier()
from_file.load_model('model_control')
assert (from_file.predict(X_train_tfidf) == model_control.predict(X_train_tfidf)).all()

In [11]:
from_file_upd = CatBoostClassifier()
from_file_upd.load_model('model_test')
assert (from_file_upd.predict(X_train_bert) == model_test.predict(X_train_bert)).all()

## Итоги обучения

**Артефакты:**
- `model_control` — CatBoostClassifier, ветка TF-IDF (сохранена в `.cbm`)
- `model_test` — CatBoostClassifier, ветка BERT (сохранена в `.cbm`)

**Быстрые оффлайн-результаты (ROC-AUC):**
- Контрольная: Train ≈ 0.6896, Test ≈ 0.6426  
- Тестовая:   Train ≈ 0.6957, Test ≈ 0.6583

> Напоминание: для задачи рекомендаций в LMS целевая метрика — **HitRate@5**.  
> Рост ROC-AUC не гарантирует рост HitRate@5, поэтому качество валидируется чекером.

**Готовность к прод-интеграции:**
- Формат сохранения `.cbm` совместим с загрузкой через `CatBoostClassifier().load_model(...)`.
- В сервисе используем отдельные имена файлов **`model_control`** и **`model_test`** и функцию `get_model_path(...)` (под пути LMS).
- Проверено: после загрузки модели предсказывают идентичнои.

**Следующий шаг:** интеграция обеих моделей в FastAPI-сервис, A/B-разбиение пользователей (md5 + соль), логирование группы и возврат топ-5 рекомендаций.
